In [1]:
from data.image_pair_dataset import Paired_Dataset
from torchvision.transforms import transforms


data_dirs = ['./training_data/DIV2K_train_HR', './training_data/Flickr2K']
data = Paired_Dataset(image_directories=data_dirs, scale=2, patch_size=(500, 500))

hr, lr = data[470]
print(hr.shape, lr.shape)

to_pil = transforms.ToPILImage()

to_pil(hr).show()
to_pil(lr).show()

torch.Size([3, 1000, 1000]) torch.Size([3, 500, 500])


In [2]:
from data.get_loader import get_loader
from torchvision.transforms import transforms

data_dirs = ['./training_data/DIV2K_train_HR', './training_data/Flickr2K']
prefetcher = get_loader(image_directories=data_dirs,
                        scale=4,
                        patch_size=(100, 100),
                        batch_size=1,
                        device=None,
                        num_workers=2,
                        pin_memory=False)


batch = prefetcher.next()
lr, hr = batch
print(lr.shape, hr.shape)
lr = lr.squeeze(0)
hr = hr.squeeze(0)
print(lr.shape, hr.shape)

to_pil = transforms.ToPILImage()

to_pil(hr).show()
to_pil(lr).show()

torch.Size([1, 3, 100, 100]) torch.Size([1, 3, 400, 400])
torch.Size([3, 100, 100]) torch.Size([3, 400, 400])


In [2]:
# from train.trainer import Trainer

# trainer = Trainer('train_yamls/RWVKSR_2X.yaml')
# # trainer.train()

In [26]:
from PIL import Image
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF

img = Image.open('training_data/test.png').convert('RGB')
img = transforms.ToTensor()(img).unsqueeze(0)
# pooled = nn.AvgPool2d(4)(img)
pooled = TF.gaussian_blur(img, kernel_size=49, sigma=1.5)
pooled = nn.Upsample(size=(img.shape[2], img.shape[3]), mode='bilinear', align_corners=False)(pooled)

hfd = img - pooled
# hfd = hfd.clamp(0, 1)
hfd = transforms.ToPILImage()(hfd.squeeze(0))
hfd.show()




In [21]:
import torch

model_file = torch.load('checkpoints/2X_RWKV4_6SR/iteration_10000.pt', map_location='cpu')
state_dict = model_file['model_state_dict']

total_params = sum(param.numel() for param in state_dict.values())

print(f"number of model parameters: {total_params/1e6}M")
print(f"{'name':90s} {'mean':>12s} {'std':>12s} {'min':>12s} {'max':>12s}")
for name, t in state_dict.items():
    # if 'high_pass_weight' in name:
    if 'shift_func' in name:
            print(f'{name:90s} {t.mean():>12f} {t.std():>12f} {t.min():>12f} {t.max():>12f}')


C:\Users\farns\AppData\Local\Temp\ipykernel_30840\3361789215.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_file = torch.load('checkpoints/2X_RWKV4_6SR/iteration_

number of model parameters: 0.992268M
name                                                                                               mean          std          min          max
layers.0.residual_group.blocks.0.RWKV.att.shift_func.sw1                                       0.535875     0.304617    -0.017982     1.001630
layers.0.residual_group.blocks.0.RWKV.att.shift_func.sw2                                       0.459471     0.287048    -0.003811     1.020562
layers.0.residual_group.blocks.0.RWKV.att.shift_func.sw3                                       0.529678     0.324245    -0.003568     1.036391
layers.0.residual_group.blocks.0.RWKV.ffn.shift_func.alpha                                     0.382154     0.340377     0.088807     0.755360
layers.0.residual_group.blocks.0.RWKV.ffn.shift_func.dconv1.weight                            -0.015819     0.187278    -0.374965     0.339564
layers.0.residual_group.blocks.0.RWKV.ffn.shift_func.dconv2.weight                            -0.019041 